In [1]:
%run _init_.py

C:\Users\m\4차4차\speed\ml\_init_.py:26: SyntaxWarning: invalid escape sequence '\W'
  font_path = "c:\Windows\Fonts\malgun.ttf"


pandas: pd 2.2.2
numpy : np 2.1.0
matplotlib : mpl 3.9.2
seaborn : sns 0.13.2
sklearn : skl 1.5.1


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor, 
                              AdaBoostRegressor, ExtraTreesRegressor, 
                              VotingRegressor, StackingRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np

In [5]:
ml = pd.read_csv('top10.csv')

### 부스팅

#### 기본

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 설정 (최적 파라미터 사용)
gbm = GradientBoostingRegressor(
    subsample=0.9,
    n_estimators=900,
    min_samples_split=2,
    min_samples_leaf=4,
    max_features='log2',
    max_depth=3,
    learning_rate=0.01
)

xgb_model = xgb.XGBRegressor(
    subsample=0.9,
    n_estimators=500,
    min_child_weight=1,
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.8
)

lgb_model = lgb.LGBMRegressor(
    subsample=0.8,
    num_leaves=31,
    n_estimators=500,
    min_child_samples=20,
    max_depth=30,
    learning_rate=0.01,
    colsample_bytree=0.9
)

# 모델 학습
gbm.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

# 개별 모델 예측
gbm_train_pred = gbm.predict(X_train)
xgb_train_pred = xgb_model.predict(X_train)
lgb_train_pred = lgb_model.predict(X_train)

gbm_test_pred = gbm.predict(X_test)
xgb_test_pred = xgb_model.predict(X_test)
lgb_test_pred = lgb_model.predict(X_test)

# 앙상블 예측
ensemble_train_pred = (gbm_train_pred + xgb_train_pred + lgb_train_pred) / 3
ensemble_test_pred = (gbm_test_pred + xgb_test_pred + lgb_test_pred) / 3

# 성능 평가
print("Individual Model R² Scores:")
print(f"Gradient Boosting Regressor Train R²: {r2_score(y_train, gbm_train_pred):.4f}")
print(f"Gradient Boosting Regressor Test R²: {r2_score(y_test, gbm_test_pred):.4f}")
print(f"XGBoost Regressor Train R²: {r2_score(y_train, xgb_train_pred):.4f}")
print(f"XGBoost Regressor Test R²: {r2_score(y_test, xgb_test_pred):.4f}")
print(f"LightGBM Regressor Train R²: {r2_score(y_train, lgb_train_pred):.4f}")
print(f"LightGBM Regressor Test R²: {r2_score(y_test, lgb_test_pred):.4f}")

print("\nEnsemble Model R² Scores:")
print(f"Ensemble Train R²: {r2_score(y_train, ensemble_train_pred):.4f}")
print(f"Ensemble Test R²: {r2_score(y_test, ensemble_test_pred):.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 1167, number of used features: 10
[LightGBM] [Info] Start training from score 12.025786
Individual Model R² Scores:
Gradient Boosting Regressor Train R²: 0.8914
Gradient Boosting Regressor Test R²: 0.8620
XGBoost Regressor Train R²: 0.8826
XGBoost Regressor Test R²: 0.8568
LightGBM Regressor Train R²: 0.9194
LightGBM Regressor Test R²: 0.8419

Ensemble Model R² Scores:
Ensemble Train R²: 0.9023
Ensemble Test R²: 0.8577


#### 2개모델

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 설정 (최적 파라미터 사용)
gbm = GradientBoostingRegressor(
    subsample=0.9,
    n_estimators=900,
    min_samples_split=2,
    min_samples_leaf=4,
    max_features='log2',
    max_depth=3,
    learning_rate=0.01
)

xgb_model = xgb.XGBRegressor(
    subsample=0.9,
    n_estimators=500,
    min_child_weight=1,
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.8
)

# 메타 모델 설정
meta_model = LinearRegression()

# 기본 모델 학습
gbm.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# 기본 모델 예측
gbm_train_pred = gbm.predict(X_train)
xgb_train_pred = xgb_model.predict(X_train)

gbm_test_pred = gbm.predict(X_test)
xgb_test_pred = xgb_model.predict(X_test)

# 기본 모델 예측을 새로운 특성으로 사용
X_train_meta = np.column_stack((gbm_train_pred, xgb_train_pred))
X_test_meta = np.column_stack((gbm_test_pred, xgb_test_pred))

# 메타 모델 학습
meta_model.fit(X_train_meta, y_train)

# 앙상블 모델 예측
ensemble_train_pred = meta_model.predict(X_train_meta)
ensemble_test_pred = meta_model.predict(X_test_meta)

# 성능 평가
print("Individual Model R² Scores:")
print(f"Gradient Boosting Regressor Train R²: {r2_score(y_train, gbm_train_pred):.4f}")
print(f"Gradient Boosting Regressor Test R²: {r2_score(y_test, gbm_test_pred):.4f}")
print(f"XGBoost Regressor Train R²: {r2_score(y_train, xgb_train_pred):.4f}")
print(f"XGBoost Regressor Test R²: {r2_score(y_test, xgb_test_pred):.4f}")

print("\nStacking Ensemble Model R² Scores:")
print(f"Ensemble Train R²: {r2_score(y_train, ensemble_train_pred):.4f}")
print(f"Ensemble Test R²: {r2_score(y_test, ensemble_test_pred):.4f}")


Individual Model R² Scores:
Gradient Boosting Regressor Train R²: 0.8905
Gradient Boosting Regressor Test R²: 0.8624
XGBoost Regressor Train R²: 0.8826
XGBoost Regressor Test R²: 0.8568

Stacking Ensemble Model R² Scores:
Ensemble Train R²: 0.8922
Ensemble Test R²: 0.8607


#### 하이퍼 파라미터 랜덤 2개모델 사용

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid_gbm = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9]
}

# 모델 학습 및 평가
def train_and_evaluate(param_grid_gbm, param_grid_xgb):
    best_score = -np.inf
    best_params = None
    best_gbm = None
    best_xgb = None
    
    # Gradient Boosting Regressor
    for gbm_params in ParameterGrid(param_grid_gbm):
        gbm = GradientBoostingRegressor(**gbm_params)
        gbm.fit(X_train, y_train)
        
        # XGBoost Regressor
        for xgb_params in ParameterGrid(param_grid_xgb):
            xgb_model = xgb.XGBRegressor(**xgb_params)
            xgb_model.fit(X_train, y_train)
            
            # 기본 모델 예측
            gbm_train_pred = gbm.predict(X_train)
            xgb_train_pred = xgb_model.predict(X_train)
            gbm_test_pred = gbm.predict(X_test)
            xgb_test_pred = xgb_model.predict(X_test)
            
            # 기본 모델 예측을 새로운 특성으로 사용
            X_train_meta = np.column_stack((gbm_train_pred, xgb_train_pred))
            X_test_meta = np.column_stack((gbm_test_pred, xgb_test_pred))
            
            # 메타 모델 학습
            meta_model = LinearRegression()
            meta_model.fit(X_train_meta, y_train)
            
            # 앙상블 모델 예측
            ensemble_train_pred = meta_model.predict(X_train_meta)
            ensemble_test_pred = meta_model.predict(X_test_meta)
            
            # 성능 평가
            train_score = r2_score(y_train, ensemble_train_pred)
            test_score = r2_score(y_test, ensemble_test_pred)
            
            # 가장 좋은 모델 저장
            if test_score > best_score:
                best_score = test_score
                best_params = (gbm_params, xgb_params)
                best_gbm = gbm
                best_xgb = xgb_model
                
    return best_gbm, best_xgb, best_params, best_score

# 최적의 모델과 파라미터 찾기
best_gbm, best_xgb, best_params, best_score = train_and_evaluate(param_grid_gbm, param_grid_xgb)

# 결과 출력
print(f"Best Parameters:\nGradient Boosting Regressor: {best_params[0]}\nXGBoost Regressor: {best_params[1]}")
print(f"Best Test R² Score: {best_score:.4f}")

# 최적 모델의 성능 평가
gbm_train_pred = best_gbm.predict(X_train)
xgb_train_pred = best_xgb.predict(X_train)
gbm_test_pred = best_gbm.predict(X_test)
xgb_test_pred = best_xgb.predict(X_test)

X_train_meta = np.column_stack((gbm_train_pred, xgb_train_pred))
X_test_meta = np.column_stack((gbm_test_pred, xgb_test_pred))

meta_model = LinearRegression()
meta_model.fit(X_train_meta, y_train)

ensemble_train_pred = meta_model.predict(X_train_meta)
ensemble_test_pred = meta_model.predict(X_test_meta)

print("\nStacking Ensemble Model R² Scores:")
print(f"Ensemble Train R²: {r2_score(y_train, ensemble_train_pred):.4f}")
print(f"Ensemble Test R²: {r2_score(y_test, ensemble_test_pred):.4f}")


Best Parameters:
Gradient Boosting Regressor: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}
XGBoost Regressor: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Best Test R² Score: 0.8633

Stacking Ensemble Model R² Scores:
Ensemble Train R²: 0.9025
Ensemble Test R²: 0.8633


### 배깅

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.utils import resample
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 설정 (최적 파라미터 사용)
gbm = GradientBoostingRegressor(
    subsample=0.9,
    n_estimators=900,
    min_samples_split=2,
    min_samples_leaf=4,
    max_features='log2',
    max_depth=3,
    learning_rate=0.01
)

xgb_model = xgb.XGBRegressor(
    subsample=0.9,
    n_estimators=500,
    min_child_weight=1,
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.8
)

lgb_model = lgb.LGBMRegressor(
    subsample=0.8,
    num_leaves=31,
    n_estimators=500,
    min_child_samples=20,
    max_depth=30,
    learning_rate=0.01,
    colsample_bytree=0.9
)

# 배깅을 위한 모델 정의
def bagging_predict(models, X_train, y_train, X_test):
    predictions_train = np.zeros((X_train.shape[0], len(models)))
    predictions_test = np.zeros((X_test.shape[0], len(models)))
    
    for i, model in enumerate(models):
        # 데이터 샘플링
        X_resampled, y_resampled = resample(X_train, y_train, random_state=42)
        model.fit(X_resampled, y_resampled)
        
        # 예측
        predictions_train[:, i] = model.predict(X_train)
        predictions_test[:, i] = model.predict(X_test)
        
    # 앙상블 예측
    ensemble_train_pred = np.mean(predictions_train, axis=1)
    ensemble_test_pred = np.mean(predictions_test, axis=1)
    
    return ensemble_train_pred, ensemble_test_pred

# 모델 리스트
models = [gbm, xgb_model, lgb_model]

# 배깅 예측
ensemble_train_pred, ensemble_test_pred = bagging_predict(models, X_train, y_train, X_test)

# 개별 모델 예측
gbm.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

gbm_train_pred = gbm.predict(X_train)
xgb_train_pred = xgb_model.predict(X_train)
lgb_train_pred = lgb_model.predict(X_train)

gbm_test_pred = gbm.predict(X_test)
xgb_test_pred = xgb_model.predict(X_test)
lgb_test_pred = lgb_model.predict(X_test)

# 성능 평가
print("Individual Model R² Scores:")
print(f"Gradient Boosting Regressor Train R²: {r2_score(y_train, gbm_train_pred):.4f}")
print(f"Gradient Boosting Regressor Test R²: {r2_score(y_test, gbm_test_pred):.4f}")
print(f"XGBoost Regressor Train R²: {r2_score(y_train, xgb_train_pred):.4f}")
print(f"XGBoost Regressor Test R²: {r2_score(y_test, xgb_test_pred):.4f}")
print(f"LightGBM Regressor Train R²: {r2_score(y_train, lgb_train_pred):.4f}")
print(f"LightGBM Regressor Test R²: {r2_score(y_test, lgb_test_pred):.4f}")

print("\nBagging Ensemble Model R² Scores:")
print(f"Ensemble Train R²: {r2_score(y_train, ensemble_train_pred):.4f}")
print(f"Ensemble Test R²: {r2_score(y_test, ensemble_test_pred):.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1035
[LightGBM] [Info] Number of data points in the train set: 1167, number of used features: 10
[LightGBM] [Info] Start training from score 12.009035
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 1167, number of used features: 10
[LightGBM] [Info] Start training from score 12.025786
Individual Model R² Scores:
Gradient Boosting Regressor Train R²: 0.8913
Gradient Boosting Regressor Test R²: 0.8602
XGBoost Regressor Train R²: 0.8826
XGBoost Regressor Test R²: 0.8568
LightGBM Regressor Train R²: 0.9194
LightGBM Regressor Test R²: 0.8419

Bagging Ensemble Model R² Scores:
Ensemble Train R²: 0.890

#### 두개 적용

### 스태킹

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 설정 (최적 파라미터 사용)
gbm = GradientBoostingRegressor(
    subsample=0.9,
    n_estimators=900,
    min_samples_split=2,
    min_samples_leaf=4,
    max_features='log2',
    max_depth=3,
    learning_rate=0.01
)

xgb_model = xgb.XGBRegressor(
    subsample=0.9,
    n_estimators=500,
    min_child_weight=1,
    max_depth=3,
    learning_rate=0.01,
    colsample_bytree=0.8
)

lgb_model = lgb.LGBMRegressor(
    subsample=0.8,
    num_leaves=31,
    n_estimators=500,
    min_child_samples=20,
    max_depth=30,
    learning_rate=0.01,
    colsample_bytree=0.9
)

# 메타 모델 설정
meta_model = LinearRegression()

# 기본 모델 학습
gbm.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

# 기본 모델 예측
gbm_train_pred = gbm.predict(X_train)
xgb_train_pred = xgb_model.predict(X_train)
lgb_train_pred = lgb_model.predict(X_train)

gbm_test_pred = gbm.predict(X_test)
xgb_test_pred = xgb_model.predict(X_test)
lgb_test_pred = lgb_model.predict(X_test)

# 기본 모델 예측을 새로운 특성으로 사용
X_train_meta = np.column_stack((gbm_train_pred, xgb_train_pred, lgb_train_pred))
X_test_meta = np.column_stack((gbm_test_pred, xgb_test_pred, lgb_test_pred))

# 메타 모델 학습
meta_model.fit(X_train_meta, y_train)

# 앙상블 모델 예측
ensemble_train_pred = meta_model.predict(X_train_meta)
ensemble_test_pred = meta_model.predict(X_test_meta)

# 성능 평가
print("Individual Model R² Scores:")
print(f"Gradient Boosting Regressor Train R²: {r2_score(y_train, gbm_train_pred):.4f}")
print(f"Gradient Boosting Regressor Test R²: {r2_score(y_test, gbm_test_pred):.4f}")
print(f"XGBoost Regressor Train R²: {r2_score(y_train, xgb_train_pred):.4f}")
print(f"XGBoost Regressor Test R²: {r2_score(y_test, xgb_test_pred):.4f}")
print(f"LightGBM Regressor Train R²: {r2_score(y_train, lgb_train_pred):.4f}")
print(f"LightGBM Regressor Test R²: {r2_score(y_test, lgb_test_pred):.4f}")

print("\nStacking Ensemble Model R² Scores:")
print(f"Ensemble Train R²: {r2_score(y_train, ensemble_train_pred):.4f}")
print(f"Ensemble Test R²: {r2_score(y_test, ensemble_test_pred):.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1062
[LightGBM] [Info] Number of data points in the train set: 1167, number of used features: 10
[LightGBM] [Info] Start training from score 12.025786
Individual Model R² Scores:
Gradient Boosting Regressor Train R²: 0.8908
Gradient Boosting Regressor Test R²: 0.8607
XGBoost Regressor Train R²: 0.8826
XGBoost Regressor Test R²: 0.8568
LightGBM Regressor Train R²: 0.9194
LightGBM Regressor Test R²: 0.8419

Stacking Ensemble Model R² Scores:
Ensemble Train R²: 0.9289
Ensemble Test R²: 0.8310


In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import xgboost as xgb

# 데이터 로드
ml = pd.read_csv('top10.csv')

# 피처와 타겟 변수 설정
X = ml.drop('SalePrice', axis=1)
y = ml['SalePrice']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 설정 (랜덤서치에서 사용할)
param_dist_gbm = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_dist_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Gradient Boosting Regressor와 XGBoost Regressor의 RandomizedSearchCV 설정
gbm = GradientBoostingRegressor(random_state=42)
gbm_random = RandomizedSearchCV(estimator=gbm, param_distributions=param_dist_gbm, 
                                n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)

xgb_model = xgb.XGBRegressor(random_state=42)
xgb_random = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist_xgb, 
                                n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)

# RandomizedSearchCV로 최적의 모델 찾기
gbm_random.fit(X_train, y_train)
xgb_random.fit(X_train, y_train)

# 최적의 모델 출력
print(f"Best GBM Parameters: {gbm_random.best_params_}")
print(f"Best XGB Parameters: {xgb_random.best_params_}")

# 최적 모델
best_gbm = gbm_random.best_estimator_
best_xgb = xgb_random.best_estimator_

# 스태킹 앙상블 구성 (기본 모델: GBM, XGB / 메타 모델: Ridge)
stacking_model = StackingRegressor(
    estimators=[('gbm', best_gbm), ('xgb', best_xgb)],
    final_estimator=Ridge()
)

# 스태킹 모델 학습
stacking_model.fit(X_train, y_train)

# 예측 및 성능 평가
train_pred = stacking_model.predict(X_train)
test_pred = stacking_model.predict(X_test)

# 성능 평가
print("\nStacking Model Performance:")
print(f"Train R²: {r2_score(y_train, train_pred):.4f}")
print(f"Test R²: {r2_score(y_test, test_pred):.4f}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best GBM Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 3, 'learning_rate': 0.2}
Best XGB Parameters: {'subsample': 0.9, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

Stacking Model Performance:
Train R²: 0.9294
Test R²: 0.8491
